In [2]:
# import webdriver
import urllib
from IPython.display import Image, display
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
# email sending
import smtplib
from email.mime.multipart import MIMEMultipart
from email.utils import make_msgid
import email.mime.application
from email.mime.text import MIMEText
import mimetypes
import pandas as pd
import csv
import math

# scrape data in lists- myGSOC()
# store data from lists to CSVs locally- storeScrapeData(orgs, tools)
# read csv, inputs
# search language in dictionary return orginisation name, no repetitions use set to store
# csv from set
# email

def storeScrapeData(orgs, tools, url):
    
    #orgs = np.array(orgs)
    #tools = np.array(tools)
    pd.DataFrame(orgs).to_csv("orgs.csv", header=None, index=None)
    pd.DataFrame(tools).to_csv("tools.csv", header=None, index=None)
    pd.DataFrame(url).to_csv("url.csv", header=None, index=None)

def myGSOC():
    # browser options
    options = Options()
    options.add_argument('--headless')
    # create webdriver object
    driver = webdriver.Chrome(options=options)
    URL = "https://summerofcode.withgoogle.com/archive/2020/organizations/"
    try:
        driver.get(URL)
    except:
        print("Invalid URL. Try again :(")
        return
    # class name for all organisations is organization-card__name font-black-54
    # the xpath for each differs by only 1 division ranging from 1-199, meaning total of 199 organisations
    orgs = []
    tools = [[]]
    url = []
    elements = driver.find_elements_by_class_name("organization-card__name")
    for i in range(len(elements)):
        orgs.append(elements[i].text)
        elements[i].click() # click on organisation
        url.append(driver.current_url)
        thisTools = driver.find_elements_by_class_name("organization__tag--technology") #find all technologies needed
        for j in range(len(thisTools)):
            tools[i].append(thisTools[j].text) # store these j elements in index i
        tools.append([]) # created new index i in the list
        driver.back()
        # since data is lost
        elements = driver.find_elements_by_class_name("organization-card__name")
    # because 1 extra got added in the last iteration
    tools.pop()
    storeScrapeData(orgs, tools, url)
    # now that data is stored locally we not not scrape again and again


emailId = input("Enter your email-id: ")
password = input("\nEnter password: ")
# tech stack available
print("\nRunning the Web Scraper...")

# run the next liine only 1 time
#myGSOC()

orgs = pd.read_csv('orgs.csv', header=None)
tools = pd.read_csv('tools.csv', header=None)
url = pd.read_csv('url.csv', header=None)
availableTools = set()
for i in range(len(tools)):
    for j in range(len(tools.loc[i])):
        if not (pd.isnull(tools.loc[i][j])):
            availableTools.add(tools.loc[i][j])
print("\nAvailable Tech Stack: ")
for t in availableTools:
    print(t, end=", ")
print("\n")
myTools = []
addTech = ''
while addTech != 'exit':
    addTech = input("\nEnter What you know from the above tech stack. Type 'exit' when done: ").lower()
    if addTech in availableTools:
        myTools.append(addTech)
    else:
        print('\nThis tool is not from the list shown. Please select a tool from the list that you know!')
if not myTools:
    print("\nSorry, please learn a tool. Your List is empty. Terminating...")
else:
    print("\nLooking up for Organisations based on the tools you know...")
    
    # lookup each entry from myTools in tools dataframe
    pos = list()
    for t in myTools:
        # Get bool dataframe with True at positions where the given value exists
        result = tools.isin([t])
        # Get list of columns that contains the value
        seriesObj = result.any()
        columnNames = list(seriesObj[seriesObj == True].index)
        # Iterate over list of columns and fetch the rows indexes where value exists
        for col in columnNames:
            rows = list(result[col][result[col] == True].index)
            for row in rows:
                pos.append((row, col))
    
    # remove repetition in the values obtained
    index = set()
    for i in range(len(pos)):
        index.add(pos[i][0])
        #print(pos[i])select
    print("\nFound "+str(len(index))+" results.")
    
    # selecting only indices that are in index array from dataframes
    orgs = orgs[orgs.index.isin(index)]
    url = url[url.index.isin(index)]
    tools = tools[tools.index.isin(index)]
    # merging all columns into 1 column
    tools['combined'] = tools[tools.columns[0:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
    # concat the dataframes into 1 single dataframe
    final = pd.concat([orgs, url, tools['combined']], axis=1)
    final.to_csv("GSOC_Organisations.csv", header=None, index=None)
    
    print("\nInitiating Email...")
    mail_content = '''<pre>Hello User!<br>
We found %s GSOC orginisations based on the choices entered by you.<br>
Please refer to the attachment for the details.
    </pre>'''%(str(len(index)))
    #The mail addresses and password
    sender_address = emailId
    sender_pass = password
    receiver_address = emailId
    #Setup the MIME
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address
    message['Subject'] = 'GSOC Organisations Found!'   #The subject line
    #The body and the attachments for the mail
    message.attach(MIMEText(mail_content, 'html'))

    # Attach csv file
    filename='GSOC_Organisations.csv'
    fp=open(filename,'rb')
    att = email.mime.application.MIMEApplication(fp.read(),_subtype="csv")
    fp.close()
    att.add_header('Content-Disposition','attachment',filename="GSOC_Organisations.csv")
    message.attach(att)

    #Create SMTP session for sending the mail
    session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
    session.starttls() #enable security
    session.login(sender_address, sender_pass) #login with mail_id and password
    text = message.as_string()
    session.sendmail(sender_address, receiver_address, text)
    session.quit()
    print('Mail Sent')

KeyboardInterrupt: Interrupted by user